**A1.4A.17** - Does the system support small data techniques (e.g active learning, few-shot learning, similarity-based completion, curriculum learning)? If yes, please elaborate.


# 1. Active Learning

Active learning is a process where the algorithm is able to interactively query the user (or some other information source) to obtain the desired outputs at new data points. The principle behind active learning is that if a learning algorithm can choose the data it wants to learn from, it can perform better than traditional methods with substantially less data for training.

In [3]:
!pip install git+https://github.com/modAL-python/modAL.git

  Cloning https://github.com/modAL-python/modAL.git to /tmp/1000730000/pip-req-build-t62tu38u
  Running command git clone --filter=blob:none --quiet https://github.com/modAL-python/modAL.git /tmp/1000730000/pip-req-build-t62tu38u
  Resolved https://github.com/modAL-python/modAL.git to commit bba6f6fd00dbb862b1e09259b78caf6cffa2e755
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 4.8 MB/s eta 0:00:00
  Created wheel for modAL-python: filename=modAL_python-0.4.2-py3-none-any.whl size=32673 sha256=b3d1b8ecd6adb54f6d7ec5d6b4f38a5be8330edfbe562e8bdbb34fa012070204
  Stored in directory: /tmp/1000730000/pip-ephem-wheel-cache-d_4i3zox/wheels/d9/fb/59/7deb61b460c1c36394cd093758986ff7d36f71352dcb2e02c5
Successfully built modAL-python


#### Active Learning with a Random Forest Classifier on the Digits Datas

This Python script is implementing an active learning procedure using a random forest classifier and the scikit-learn digits dataset.

The script starts by loading the digits dataset, then it randomly selects an initial subset of data points as the initial training set for the active learner. The rest of the data points are set aside as an 'unlabeled' pool.

The active learner, built on top of a random forest classifier, is then initialized with the initial training set. It's performance is evaluated and printed on the console.

The script then enters a loop where it actively selects and queries data points from the 'unlabeled' pool for labeling. After each query, the learner updates its model with the new instance and removes it from the 'unlabeled' pool.

This process is repeated for a specified number of queries. After this active learning process, the learner's performance is evaluated again and printed on the console, showing the accuracy improvement after actively learning from additional instances.

In [6]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from modAL.models import ActiveLearner
import numpy as np

# loading the digits dataset
digits = load_digits()

# initial training data
n_initial = 100
initial_idx = np.random.choice(range(len(digits.data)), size=n_initial, replace=False)
X_initial, y_initial = digits.data[initial_idx], digits.target[initial_idx]

# generating the 'unlabeled' pool
X_pool = np.delete(digits.data, initial_idx, axis=0)
y_pool = np.delete(digits.target, initial_idx, axis=0)

# initializing the learner
learner = ActiveLearner(
    estimator=RandomForestClassifier(),
    X_training=X_initial, y_training=y_initial
)

# initial accuracy
accuracy = learner.score(digits.data, digits.target)
print('Initial accuracy: %f' % (accuracy))


# pool-based sampling
n_queries = 100
for idx in range(n_queries):
    query_idx, query_instance = learner.query(X_pool)
    learner.teach(
        X=X_pool[query_idx].reshape(1, -1),
        y=y_pool[query_idx].reshape(1, )
    )
    # remove queried instance from pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)
    
# final accuracy
accuracy = learner.score(digits.data, digits.target)
print('Accuracy after %d queries: %f' % (n_queries, accuracy))


Initial accuracy: 0.857540
Accuracy after 100 queries: 0.969393


# 2. Few Shot Learning

Few-shot learning refers to designing machine learning models that can learn valuable information from a small number of examples, a crucial feature for environments with limited data. It's at the heart of recent large language models like GPT, facilitating rapid adaptation to new tasks based on prior knowledge.

In this section, we'll implement few-shot learning using the pre-trained MobileNetV2 model, a popular architecture known for its efficiency and performance in computer vision tasks. We'll fine-tune MobileNetV2 on a small novel dataset, showcasing its potential for tasks with scarce data.



In [40]:
# Function to download and extract dataset
def download_extract_dataset(url, local_directory, dir_train_ohara, dir_test_ohara, 
                             dir_train_misterlincoln, dir_test_misterlincoln, show_more=False):
    import os
    import requests
    from zipfile import ZipFile
    from PIL import Image
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg

    response = requests.get(url)
    file_name = os.path.join(local_directory, "rose_training.zip")

    # Save the content in a .zip file
    with open(file_name, "wb") as file:
        file.write(response.content)

    # Unzip the file
    with ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall(local_directory)

    # Remove the zip file
    os.remove(file_name)

    # Define the image file names with correct paths
    train_ohara = [os.path.join(dir_train_ohara, file) for file in os.listdir(dir_train_ohara) ]
    test_ohara = [os.path.join(dir_test_ohara, file) for file in os.listdir(dir_test_ohara)]
    train_misterlincoln = [os.path.join(dir_train_misterlincoln, file) for file in os.listdir(dir_train_misterlincoln) ]
    test_misterlincoln = [os.path.join(dir_test_misterlincoln, file) for file in os.listdir(dir_test_misterlincoln)]

    # Combine all image lists
    all_images = train_ohara  + train_misterlincoln + test_ohara + test_misterlincoln

    if show_more:
        all_images = [img for img in all_images if 'more' in os.path.basename(img)]

    # Define number of columns (assuming each set has 5 images for training and 2 for testing)
    columns = 5

    # Create subplot to display the images
    for i, image_name in enumerate(all_images):
        img = mpimg.imread(image_name)
        plt.subplot(len(all_images) // columns + 1, columns, i + 1)
        plt.imshow(img)
        plt.axis('off')
        if show_more:
            plt.title(os.path.splitext(os.path.basename(image_name).replace("more_",""))[0])
        else:            
            plt.title(os.path.splitext(os.path.basename(image_name))[0])
        
    plt.show()


In [26]:
url = "https://github.com/tianhuat/share_files/raw/main/rose_training.zip"
local_directory = "."
dir_train_ohara = "rose_training/train/ohara"
dir_test_ohara = "rose_training/test/ohara"
dir_train_misterlincoln = "rose_training/train/misterlincoln"
dir_test_misterlincoln = "rose_training/test/misterlincoln"

download_extract_dataset(url, local_directory, dir_train_ohara, dir_test_ohara, dir_train_misterlincoln, dir_test_misterlincoln)

In [10]:
pip install --upgrade  tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 102.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 79.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 68.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 94.5 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 98.4 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 49.1 MB/s eta 0:00:00


  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.1
    Uninstalling numpy-1.25.1:
      Successfully uninstalled numpy-1.25.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling 

In [28]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
# load pretrained model
base_model = tf.keras.applications.MobileNetV2(input_shape=(160, 160, 3),
                                               include_top=False,
                                               weights='imagenet')

# freeze base model
base_model.trainable = False

# create new model on top
inputs = tf.keras.Input(shape=(160, 160, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

# compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# load image dataset
train_ds = image_dataset_from_directory('rose_training/train',
                                        labels='inferred',
                                        label_mode='binary',
                                        batch_size=32,
                                        image_size=(160, 160),
                                        shuffle=True,
                                      
                                       
                                        seed=123)

val_ds = image_dataset_from_directory('rose_training/test',
                                      labels='inferred',
                                      label_mode='binary',
                                      batch_size=32,
                                      image_size=(160, 160),
                                      shuffle=True,
                                    
                                   
                                      seed=123)

Found 10 files belonging to 2 classes.
Found 2 files belonging to 2 classes.


In [4]:
# fine-tune the model
history = model.fit(train_ds, validation_data=val_ds, epochs=20)

# evaluate the model
print("Model evaluation:")
model.evaluate(val_ds)

Found 10 files belonging to 2 classes.
Found 2 files belonging to 2 classes.
Epoch 1/20
1/1 [==============================] - 4s 4s/step - loss: 0.6146 - accuracy: 0.6000 - val_loss: 0.6028 - val_accuracy: 1.0000
Epoch 2/20
1/1 [==============================] - 0s 413ms/step - loss: 0.5357 - accuracy: 0.7000 - val_loss: 0.5612 - val_accuracy: 1.0000
Epoch 3/20
1/1 [==============================] - 0s 410ms/step - loss: 0.4711 - accuracy: 0.7000 - val_loss: 0.5295 - val_accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - 0s 443ms/step - loss: 0.4135 - accuracy: 0.7000 - val_loss: 0.5056 - val_accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 0s 412ms/step - loss: 0.3618 - accuracy: 0.8000 - val_loss: 0.4876 - val_accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 425ms/step - loss: 0.3167 - accuracy: 0.9000 - val_loss: 0.4732 - val_accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 0s 406ms/step - loss: 0.2783 - accur

[0.28023561835289, 1.0]

In [ ]:
model.save('/project_data/data_asset/few_shot.h5')

In [29]:
# load the entire model
model = tf.keras.models.load_model('/project_data/data_asset/few_shot.h5')

In [30]:
import numpy as np

# This code assumes `val_ds` is your validation dataset
for images, labels in val_ds.take(1):  # You can change the number in take() to get more batches
    predictions = model.predict(images)
    predictions = tf.sigmoid(predictions) # Apply sigmoid function to get probability from logits
    predictions = tf.round(predictions) # Round to get binary classification
    predictions = predictions.numpy().flatten()  # Flatten the predictions array

    labels = labels.numpy().flatten()  # Flatten the labels array
    
    for i in range(len(labels)):
        print(f"Actual: {labels[i]}, Predicted: {predictions[i]}")

1/1 [==============================] - 1s 1s/step
Actual: 1.0, Predicted: 1.0
Actual: 0.0, Predicted: 0.0


# 3. Curriculum Learning

### Curriculum Learning: Origins and Applications

In 2009, Yoshua Bengio and his colleagues introduced the concept of **curriculum learning** in their groundbreaking paper ["Curriculum Learning"][1]. They identified a promising pathway for enhancing machine learning: a structured learning journey that moves from straightforward to complex concepts. Inspired by the innate learning processes of humans and animals, this methodology signals its potential applicability in the realm of artificial intelligence.

### Practical Case Study: Product Line Configuration

![Curriculum Learning](https://i.stack.imgur.com/kyeaS.png)

In the context of product line configuration, we encounter multiple objectives:

- **Correctness**: Minimize feature model constraints violations.
- **Richness**: Minimize non-selected features.
- **Cost**: Keep total costs to a minimum.
- **Usage**: Minimize unused features.
- **Defects**: Minimize known flaws.

To optimize these objectives, we deploy the multi-objective evolutionary algorithm Indicator-Based Evolutionary Algorithm (IBEA).

Take, for instance, a scenario with 320 independent features providing an astonishing 2^320 configurations - a quantity exceeding the estimated number of atoms in the universe. When handling a Linux kernel with over 6000 features, we approach the challenge as follows:

1. **Traditional IBEA** struggles to produce results within an acceptable time frame.
2. **IBEA with a curriculum learning strategy of 2 initial objectives, extending to 5** [2] manages to generate 36 correct solutions in 4 hours.
3. **Error-centric IBEA employing curriculum learning** [3] impressively cuts down this time to just 40 seconds for 36 correct solutions.

This case study underscores the effectiveness of curriculum learning in facilitating swifter convergence to superior solutions by simplifying the problem at hand.

[1]: Bengio, Y., Louradour, J., Collobert, R., & Weston, J. (2009). Curriculum Learning. In *Proceedings of the 26th Annual International Conference on Machine Learning* (pp. 41-48).

[2]: Sayyad, A. S., Ingram, J., Menzies, T., & Ammar, H. (2013). Scalable product line configuration: A straw to break the camel's back. In ASE.

[3]: Tan, T. H., Xue, Y., Chen, M., Sun, J., Liu, Y., & Dong, J. S. (2015). Optimizing selection of competing features via feedback-directed evolutionary algorithms. In _Proceedings of the 2015 International Symposium on Software Testing and Analysis, ISSTA 2015, Baltimore, MD, USA, July 12-17, 2015_ (pp. 246-256).

In [33]:
url = "https://github.com/tianhuat/share_files/raw/main/rose_training.zip"
local_directory = "."
dir_train_ohara = "rose_training/train_more/ohara"
dir_test_ohara = "rose_training/test_more/ohara"
dir_train_misterlincoln = "rose_training/train_more/misterlincoln"
dir_test_misterlincoln = "rose_training/test_more/misterlincoln"

download_extract_dataset(url, local_directory, dir_train_ohara, dir_test_ohara, dir_train_misterlincoln, dir_test_misterlincoln, show_more=True)

In [34]:
# load the saved model
from tensorflow.keras.models import load_model
model = load_model('/project_data/data_asset/few_shot.h5')

# load image dataset
train_ds = image_dataset_from_directory('rose_training/train_more',
                                        labels='inferred',
                                        label_mode='binary',
                                        batch_size=32,
                                        image_size=(160, 160),
                                        shuffle=True,                                                                             
                                        seed=123)

val_ds = image_dataset_from_directory('rose_training/test_more',
                                      labels='inferred',
                                      label_mode='binary',
                                      batch_size=32,
                                      image_size=(160, 160),
                                      shuffle=True,                                                                       
                                      seed=123)

Found 20 files belonging to 2 classes.
Found 4 files belonging to 2 classes.


In [19]:
# fine-tune the model
history = model.fit(train_ds, validation_data=val_ds, epochs=40)

# evaluate the model
print("Model evaluation:")
model.evaluate(val_ds)

Found 20 files belonging to 2 classes.
Found 4 files belonging to 2 classes.
Epoch 1/40
1/1 [==============================] - 4s 4s/step - loss: 0.4277 - accuracy: 0.8500 - val_loss: 0.5733 - val_accuracy: 0.5000
Epoch 2/40
1/1 [==============================] - 1s 583ms/step - loss: 0.4143 - accuracy: 0.8500 - val_loss: 0.5504 - val_accuracy: 0.5000
Epoch 3/40
1/1 [==============================] - 1s 576ms/step - loss: 0.3946 - accuracy: 0.8500 - val_loss: 0.5251 - val_accuracy: 0.7500
Epoch 4/40
1/1 [==============================] - 1s 581ms/step - loss: 0.3710 - accuracy: 0.8500 - val_loss: 0.5001 - val_accuracy: 0.7500
Epoch 5/40
1/1 [==============================] - 1s 607ms/step - loss: 0.3459 - accuracy: 0.8500 - val_loss: 0.4778 - val_accuracy: 0.7500
Epoch 6/40
1/1 [==============================] - 1s 592ms/step - loss: 0.3214 - accuracy: 0.8500 - val_loss: 0.4596 - val_accuracy: 0.7500
Epoch 7/40
1/1 [==============================] - 1s 534ms/step - loss: 0.2989 - accur

[0.18566855788230896, 1.0]

In [20]:
model.save('/project_data/data_asset/curriculum.h5')

/opt/conda/envs/Python-3.10-Premium/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:
# load the entire model
model = tf.keras.models.load_model('/project_data/data_asset/curriculum.h5')


In [36]:
import numpy as np

# This code assumes `val_ds` is your validation dataset
for images, labels in val_ds.take(1):  # You can change the number in take() to get more batches
    predictions = model.predict(images)
    predictions = tf.sigmoid(predictions) # Apply sigmoid function to get probability from logits
    predictions = tf.round(predictions) # Round to get binary classification
    predictions = predictions.numpy().flatten()  # Flatten the predictions array

    labels = labels.numpy().flatten()  # Flatten the labels array
    
    for i in range(len(labels)):
        print(f"Actual: {labels[i]}, Predicted: {predictions[i]}")

1/1 [==============================] - 1s 998ms/step
Actual: 1.0, Predicted: 1.0
Actual: 0.0, Predicted: 0.0
Actual: 1.0, Predicted: 1.0
Actual: 0.0, Predicted: 0.0


# 4. Similarity-based completion

Previously, we explored the application of K-Nearest Neighbors (KNN) algorithm in a recommendation system, predicting reviews based on user preferences. Now, we are going to use KNN in a different context, handling missing values in a dataset through similarity-based completion.

In [39]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

# Our small dataset with missing values
# data = np.array([[1, 2], 
#                  [2, 4], 
#                  [3, np.nan], 
#                  [4, 8], 
#                  [5, np.nan]])
data = []
for i in range(1, 101):
    # If i % 3 == 2, add a np.nan, else double the value
    second_dim_value = np.nan if i % 3 == 2 else 2 * i
    data.append([i, second_dim_value])

print("Input Data:")
print(data)

# Normalize the data
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Initialize the imputer
imputer = KNNImputer(n_neighbors=3)

# Fit on the dataset and transform the data
complete_data = imputer.fit_transform(data)

# Convert back to original scale
complete_data = scaler.inverse_transform(complete_data)

print("\nKNN Imputed Data:")
print(complete_data)

Input Data:
[[1, 2], [2, nan], [3, 6], [4, 8], [5, nan], [6, 12], [7, 14], [8, nan], [9, 18], [10, 20], [11, nan], [12, 24], [13, 26], [14, nan], [15, 30], [16, 32], [17, nan], [18, 36], [19, 38], [20, nan], [21, 42], [22, 44], [23, nan], [24, 48], [25, 50], [26, nan], [27, 54], [28, 56], [29, nan], [30, 60], [31, 62], [32, nan], [33, 66], [34, 68], [35, nan], [36, 72], [37, 74], [38, nan], [39, 78], [40, 80], [41, nan], [42, 84], [43, 86], [44, nan], [45, 90], [46, 92], [47, nan], [48, 96], [49, 98], [50, nan], [51, 102], [52, 104], [53, nan], [54, 108], [55, 110], [56, nan], [57, 114], [58, 116], [59, nan], [60, 120], [61, 122], [62, nan], [63, 126], [64, 128], [65, nan], [66, 132], [67, 134], [68, nan], [69, 138], [70, 140], [71, nan], [72, 144], [73, 146], [74, nan], [75, 150], [76, 152], [77, nan], [78, 156], [79, 158], [80, nan], [81, 162], [82, 164], [83, nan], [84, 168], [85, 170], [86, nan], [87, 174], [88, 176], [89, nan], [90, 180], [91, 182], [92, nan], [93, 186], [94, 188]